[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prokaj/elte-python/blob/main/9-gyakorlat.ipynb)

# Házi feladatok

Implementáljuk a `RandomNames` osztályt.

A egyedeknek legyen egy `names` property-je, ami olvasáskor a névsort az eredeti sorrendben adja vissza, és íráskor beállítja az új névsort. A `RandomNames` legyen iterálható és az `__iter__` metódusa egy végtelen generátort adjon vissza, ami véletlen sorrendben megy végig a neveken, ha végig ért akkor újra kezdi újrasorsolt véletlen sorrendben. Az egyedek legyenek függvényszerűek is (`__call__` metódus). A függvény hívás eredménye legyen egyetlen találomra választott név a névsorból.

Emlékeztető: Egy osztályban a `@property` dekorátorral tudunk `property` mezőt létrehozni. A dekorált függvény szolgál a mező kiolvasára. Ha írhatóvá szeretnénk tenni a mezőt, akkor a `mezőnév.setter` dekorátort kell használni.

Pl.
```
class RandomNames:

    @property
    def names(self):
        pass
   
    @names.setter
    def names(self, new_value):
        pass
```
A names mező írásakor végezzen ellenőrzést a függvény. Azaz ha nem sztringekből álló sorozattal hívjuk meg, akkor dobjon `ValueError`-t. A kapott nevek első betűjét írja át nagy betűvé, a többit viszont írja át kis betűre. Figyeljünk arra, hogy `rn.names = 'Attila'` hibát kell, hogy dobjon, de a sorozat nem csak lista lehet, hanem bármi amin végig lehet iterálni!

Pl.

```
rn = RandomNames(['anDoR'])
rn.names # -> ['Andor']
rn.names = ['ALADÁR', 'elemér']
rn.names # -> ['Aladár', 'Elemér']
```

A véletlenszerű sorrend előállításához használhatjuk a `random` modul, `shuffle` függvényét. Figyeljünk arra, hogy ez elronthatja az eredeti sorrendet!

Inicializáláskor az egyed kap egy névsort. Ezt kezelje úgy, mintha names tulajdonságot írtuk volna!

Írjunk teszt függvényt! Ez ellenőrizze az inicializálást, ill. names tulajdonság írását, olvasását. Úgy is, hogy a kettő között az iterátort is használtuk.

Gondolkozzunk el azon, hogyan lehetne a véletlenszám generálást tartalmazó részeket tesztelni: iterátor, függvény hívás. Ha más nem jut eszünkbe, akkor az alábbi lehetőségek közül legalább egyet implementáljunk a teszt függvényben.

Ha $n$ nevet adtunk meg, akkor $n$-szer iterálva az iterátort mindegyik nevet pontosan egyszer kell, hogy megkapjuk.

Ha $n$ elég nagy, $n$>10, akkor két egymástól független véletlen sorrend nagyon kis valószínűséggel esik egybe.

Ha sokszor hívjuk meg az egyedet, akkor mindegyik név nagyjából $1/n$
relatív gyakorísággal kell, hogy szerepeljen. pl. ha $m$ a hívások száma, akkor $4/\sqrt{nm}$ eltérést megengedve, csak nagyon kis valószínűséggel téved a teszt.

Most és érdemes az osztály vázával és a tesztfüggvénnyel kezdeni.

In [ ]:
import random 

    
class RandomNames:
    def __init__(self, names):
        self.names = names

    def __str__(self):
        return f"{self.names}"
    
    def __repr__(self):
        return f"{type(self).__name__}({self.names})"
    
    @property
    def names(self):
        return self._names
    
    @names.setter
    def names(self, values):
        ## checking
        self._names = values
        return self.names

    def __iter__(self):
        pass        

    def __call__(self):
        pass
        

        

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest 
from pytest import raises
from collections import Counter 
from itertools import islice
from scipy.stats import chi2 
import math

def is_uniform(counts):
    n = sum(counts)
    np = n/len(counts)
    stat = sum(((v-np)**2)/np for v in counts)
    return chi2.sf(stat, df=len(counts)-1)>0.001

def test_random_names():
    names = list("abcde".upper())
    rn = RandomNames(list("abcde"))
    assert rn.names == names

    with raises(ValueError):
        rn.names = "attila"
    
    with raises(ValueError):
        rn.names = [1, "a", "aa", "bb"]

    rn.names = map(str, range(101))
    assert rn.names == [str(i).capitalize() for i in range(101)]

    rn.names = tuple("abcde")
    assert rn.names == [x.capitalize() for x in "abcde"]

    
    rn.names = [x*3 for x in "abcdefghujk"]

    name_list = rn.names.copy()

    for name in rn.names:
        assert name == name.capitalize()

def test_call():
    rn = RandomNames(x*3 for x in "abcdefghujk")

    name_list = rn.names.copy()

    for name in rn.names:
        assert name == name.capitalize()

    ### __call__ 
    for _ in range(100):
        assert rn() in rn.names 

    ## names should not change 
    assert rn.names == name_list

    ## randomness in __call__
    n = 10_000
    cnt = Counter(rn() for _ in range(n))
    assert is_uniform(list(cnt.values())) 

def test_iter():
    rn = RandomNames(x*5 for x in "abcdefghujk")

    name_list = rn.names.copy()

    ### __iter__
    assert set(islice(rn, len(rn.names))) == set(rn.names)

    it = iter(rn)
    lst1, lst2 = islice(it, len(rn.names)), islice(it, len(rn.names))

    ## names should not change
    assert rn.names == name_list
    ## each block of len(rn.names) should be a random permutation 
    assert set(lst1) == set(rn.names)
    assert set(lst2) == set(rn.names)
    assert lst1 != lst2
    
    it = iter(rn)
    lst = [list(islice(it, len(rn.names))) for _ in range(10_000)]
    cnts = [list(Counter(x).values()) for x in zip(*lst)] 
    print([(min(x), max(x)) for x in cnts])
    assert all([is_uniform(cnt) for cnt in cnts])
    


Írjuk meg a `RandomNames` osztály implementációját!

In [ ]:
import random 

def capitalize(x):
    if not isinstance(x, str):
        raise ValueError
    return x.capitalize()
    
class RandomNames:
    def __init__(self, names):
        self.names = names

    def __str__(self):
        return f"{self.names}"
    
    def __repr__(self):
        return f"{type(self).__name__}({self.names})"
    
    @property
    def names(self):
        return self._names
    
    @names.setter
    def names(self, values):
        ## checking
        if isinstance(values, str):
            raise ValueError
        self._names = [capitalize(x) for x in values]
        return self.names

    def __iter__(self):
        names = self.names.copy()
        while True:
            random.shuffle(names)
            for name in names:
                yield name
        

    def __call__(self):
        return random.choice(self._names)


In [ ]:
rn = RandomNames(x*3 for x in "abcde")
list(rn() for _ in range(10))
rn, str(rn)

## További példák dekorátorokra

A `functools` modulban számos hasznos dekorátor található

- `lru_cache`: A függvényből a memorizált változatot készíti el.
- `partial`: `g = partial(f, 1)` hatására `g(...)` ugyanaz, mint `f(1, ...)`
- `wraps`: lásd a dokumentációt.

In [ ]:
from functools import lru_cache, partial, wraps

@lru_cache()
def f(x):
    return x*x

print(f(1), f(2), f(1))
print(f.cache_info())

def add(a, b):
    return a+b

inc = partial(add, 1)
inc(10)




In [ ]:
def decor_wo_wraps(f):
    def h(*args):
        print("decorated without wraps")
        return f(*args)
    return h

def decor_wraps(f):
    @wraps(f)
    def h(*args):
        print("decorated with wraps")
        return f(*args)
    return h


In [ ]:
@decor_wo_wraps
def dummy_fn(a:int, b:int) -> int:
    """dummy fn. Nothing interesting"""
    return a+b

print(dummy_fn(1, 2))
help(dummy_fn)

In [ ]:
@decor_wraps
def dummy_fn(a:int, b:int) ->int:
    """dummy fn. Nothing interesting"""
    return a+b

print(dummy_fn(1, 2))
help(dummy_fn)

# Öröklődés

Előadáson szerepelt ehhez hasonló példa. Legyenek `Polygon`, `Rectangle`, `Square` osztályaink.
Minden négyzet téglalap és minden téglalap sokszög. Egy sokszöget a csúcsok felsorolásával adhatunk meg.
Ebből a kerület, terület kiszámolható és eldönthető, hogy a sokszög konvex-e. Ha akarjuk pl. a `Matplotlib` könyvtárral ki is rajzolhatjuk a sokszöget.

Menetközben síkvektorokkal akarunk számolni. Erre kényelmes a `numpy` könyvtárat használni.

In [1]:
import math
import numpy as np
from itertools import cycle

class Polygon:

    def __init__(self, nodes):
        self.nodes = nodes

    def area(self):
        if self.is_convex():
            nodes = np.array(self.nodes)
            nodes = nodes-nodes[0]
            area = 0
            for v1, v2 in zip(nodes[1:], nodes[2:]):
                area += abs(v1[0]*v2[1]-v1[1]*v2[0])
            return area
        else:
            raise NotImplementedError
            ## hf!

    def perimeter(self):
        total = 0
        for i, p in enumerate(self.nodes, 1):
            j = i % len(self.nodes)
            total += np.linalg.norm(self.nodes[j]-p)
        return total

    def is_convex(self):
        nodes = np.array(self.nodes + self.nodes[:1])
        edges = np.diff(nodes, axis=0)
        idx = np.array([i%len(edges) for i in range(1, len(edges)+1)])
        edges1 = edges[idx]
        sgn = np.sign((rotate90(edges) * edges1).sum(axis=1))
        return (sgn==sgn[0]).all()

    def draw(self):
        pass
    
    def __repr__(self):
        return f'{type(self).__name__}({[tuple(x) for x in self.nodes]})'


def rotate90(x):
    return np.flip(x, -1)*np.array([-1, 1])

class Rectangle(Polygon):
    
    def __init__(self, P1, P2, r):
        """
        Assuming that e is P1P2 rotated counterclockwise by 90 degree
        P1, P2, P2+r e, P1+re
        """
        v = rotate90(P2-P1)*r
        super(Rectangle, self).__init__([P1, P2, P2+v, P1+v])
        
    
class Square(Rectangle):

    def __init__(self, P1, P2):
        super(Square, self).__init__(P1, P2, 1)
        


In [ ]:
rotate90(np.array([[0, 1],[1,0]]))

In [2]:
points = np.array([[0,0], [1,1]])
s = Square(points[0], points[1])
t = Rectangle(points[0], points[1], 2)
p = Polygon(points)
for x in [s, t, p]:
    print(f"{x} kerület: {x.perimeter():.2f}, konvex: {x.is_convex()}, terület: {x.area():.2f}")
    

Square([(0, 0), (1, 1), (0, 2), (-1, 1)]) kerület: 5.66, konvex: True, terület: 4.00
Rectangle([(0, 0), (1, 1), (-1, 3), (-2, 2)]) kerület: 8.49, konvex: True, terület: 8.00
Polygon([(0, 0), (1, 1)]) kerület: 2.83, konvex: True, terület: 0.00


Vegyük észre, hogy az `__repr__` metódust csak egyszer implementáltuk, mégis minden egyednek a saját típusát írja ki!


# `Matplotlib.pyplot` könytár

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

A legtöbbet használt függvények a  `plot` és az `imshow`.

In [ ]:
plt.plot?

In [ ]:
x = np.linspace(0,10, 201)
print(x[:10])
plt.plot(x, np.sin(x))
plt.grid()
plt.show()

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/en/thumb/7/7d/Lenna_%28test_image%29.png/220px-Lenna_%28test_image%29.png" 
img = plt.imread(url)
img.shape

In [ ]:
import PIL
import urllib
img = np.array(PIL.Image.open(urllib.request.urlopen(url)))
img.shape

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
plt.plot(*np.array(s.nodes)[[0,1,2,3,0]].T)
plt.grid()
plt.axis('equal')
# plt.axis('off')


In [ ]:
def draw(self,  *args, fill=False, **kwargs):
    nodes = self.nodes + self.nodes[:1]
    x, y = np.array(nodes).T
    if fill:
        plt.fill(x, y, alpha=0.5)
    plt.plot(x, y, *args, **kwargs) 
    plt.axis('equal')

Polygon.draw = draw

In [ ]:
t.draw('o-', alpha=0.5)
s.draw(alpha=0.5, fill=True)
p.draw(alpha=0.5)
plt.show()

# Conway Game of Life


The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:

- **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
- **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell.


Írjunk egy osztályt a játékhoz, pl. az `__init__` metódus hozzalétre a megadott konfigurációnak megfelelő objektumot. Legyen egy `step` metódus, ami a rendszert a következő állapotába viszi és az `__str__` metódus pedig
valahogy ábrázolja az aktuális állapotot. 

Tegyük fel, hogy a rács amin a rendszer él, egy $n\times n$-es rács, ahol mindkét irányban ciklikusan körbemegyünk,
azaz a csúcsokat modulo $n$ tekintjük.


In [ ]:
class ConwayGoL:
    
    def __init__(self, state):
        self.state = list(state)

    def step(self):
        return self

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

In [ ]:
import random

init_state = [] ## ???
conway = ConwayGoL(init_state)

conway.step()


A játék állapotának leírásához egy $n\times n$ rács minden pontjáról tudni kell, hogy foglalt-e vagy sem.

```
n = 11
state = [[0]*n for _ in range(n)]
```

Véletlenszerű kezdeti állapot:
```
state = [[random.randint(0,1) for _ in range(n)] for _ in range(n)]
```

In [ ]:
def random_state(n, m, p):
    return [[int(random.random()<p) for _ in range(m)] for _ in range(n)]

state = random_state(11, 25, 0.2)
print(state)

Szebb megjelenítés?

In [ ]:
def as_matrix(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

print('\n'.join(''.join(map(str, line)) for line in state))


In [ ]:
symbols = "\u2b1c\u2b1b"
print('\n'.join(''.join(symbols[x] for x in line) for line in state))


In [ ]:
import matplotlib.pyplot as plt 

img = plt.matshow(state, cmap="viridis")
#img.axes.axis(False)
n, m = len(state), len(state[0])
for pos in range(0, n+1):
    img.axes.axhline(y=pos-0.5, color="white")
for pos in range(0, m+1):
    img.axes.axvline(x=pos-0.5, color="white")

plt.show()


In [ ]:

def cgol_str(self):
    symbols = "\u2b1c\u2b1b"
    return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

ConwayGoL.__str__ = cgol_str

    

In [ ]:
conway = ConwayGoL(state)
print(conway)

A `step` metódushoz ki kellene számolni egy adott csúcs foglalt szomszédainak számát `cnt`. Ha ez kész,
akkor az $i$ csúcs új állapota:

$$
    \text{state}_{t+1}[i]=
    \begin{cases}
    1 &\text{Ha $\text{cnt}[i]\in\{2,3\}$ és $\text{state}_t[i]=1$}\\
    1 &\text{Ha $\text{cnt}[i]\in\{3\}$ és $\text{state}_t[i]=0$}\\
    0 &\text{különben}
    \end{cases}
$$

In [ ]:
def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

In [ ]:
%%ipytest

def test_newstate():
    res = [0]*9
    res[2] = 1
    res[3] = 1
    assert newstate([1]*9, list(range(9))) == res
    res = [0]*9
    res[3] = 1    
    assert newstate([0]*9, list(range(9))) == res


In [ ]:
def count_neighbors(state):
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 
    n, m = len(state), len(state[0])
    return [ [sum(state[(i+di) % n][(j+dj) % m] for di, dj in strides) for j in range(m)] for i in range(n)]

def cgol_step(self):
    counts = count_neighbors(self.state)
    self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
    return self
    
ConwayGoL.step = cgol_step

In [ ]:
@classmethod
def cgol_from_random_state(cls, n, m,  p):
    return cls(random_state(n, m, p))

ConwayGoL.from_random_state=cgol_from_random_state

In [ ]:
conway = ConwayGoL.from_random_state(4, 4, 0.3)
print(conway)
print(count_neighbors(conway.state))

In [ ]:
conway = ConwayGoL.from_random_state(5, 5, 0.5)
print(conway)
print("-"*20)
print(conway.step())

Tudunk-e valami animációszerűt készíteni? Jupyter notebook-ban pl. a következő képpen lehet: 

In [ ]:
from ipywidgets import Output
from time import sleep

out = Output()
display(out)
conway = ConwayGoL.from_random_state(n=31, m=55, p=0.2)

for i in range(100):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway}")  
    sleep(0.2)
    conway.step()


Ha parancssorból dolgozunk, akkor valami ilyesmit lehetne tenni

In [ ]:
%%writefile conway.py

import random


def random_state(n, m, p):
    return [ [ int(random.random()<p) for _ in range(m) ] for _ in range(n) ]

def count_neighbors(state):
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 
    n, m = len(state), len(state[0])
    return [ [sum(state[(i+di) % n][(j+dj) % m] for di, dj in strides) for j in range(m)] for i in range(n)]

def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

class ConwayGoL:
    symbols = "\u2b1c\u2b1b"
    
    def __init__(self, state):
        self.state = list(state)
        
    def step(self):
        counts = count_neighbors(self.state)
        self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
        return self

    
    def __str__(self): 
        symbols = self.symbols
        return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"
    
    @classmethod
    def from_random_state(cls, n, m, p):
        return cls(random_state(n, m, p))

    def is_empty(self):
        return not any(any(line) for line in self.state)


def clear_terminal(n):
    print(f"{chr(27)}[{n+1}A", end="")

def main(n=11, m=25, p=0.2, nsteps=10, clear_screen=clear_terminal):
    from time import sleep
    conway = ConwayGoL.from_random_state(n, m, p)
    for i in range(nsteps+1):
        if i>0:
            clear_screen(n)
        print(f"after {i} step:")
        print(conway)  
        sleep(0.2)
        conway.step()
        if conway.is_empty():
            break
    
if __name__ == "__main__":
    main()
    

Ha valamit már megírtunk és szeretnénk használni, `import`-tal elérhető. Pl.

### Tudunk-e paramétereket adni a python scriptnek?

Amikor egy python scriptet futtatunk, a parancssor (amivel a futást indítottuk) a `sys` modul `argv` változójában érhető el.

In [ ]:
import sys
sys.argv

In [ ]:
! python -c 'import sys; print(sys.argv)' -alma


Egy nagyon egyszerű megoldás, ha minden opciónak a neve a paraméter amit beállít és egyenlőségjel után az értéke:
pl. n=11 m=25 nstep=10 p=0.2

In [ ]:
cmdline = "conway.py -n=11 -m=25 -nstep=10 -p=0.2"
argv = cmdline.split()
params =[param.split("=") for param in argv[1:]]
params

Minden paraméterről tudni kellene, hogy milyen típusú!

In [ ]:
param_types={'-n': int, '-m': int, '-nstep': int, '-p': float}
params = {k.replace("-",""): param_types[k](v)  for k, v in (param.split("=") for param in argv[1:])}
params

Ezek után a `main` függvényt a megadott paraméterekkel meg tudjuk hívni:

```
    main(**params)
```
Mi van a `default` értékekkel, `help`-pel stb.

Ezeket mind meg tudnánk írni, de nem kell. Van kész megoldás `python`-ban.

Az `argparse` könvytár mindent megcsinál, ami nekünk kell.

In [ ]:
import argparse

help(argparse)

A `conway.py` file végét cseréljük le erre.
```
if __name__=="__main__":

    import argparse
    
    parser = argparse.ArgumentParser(description='Conways Game of Life')

    parser.add_argument(
        '-n', '--nrows', 
        type=int, 
        default=11, 
        help='number of rows'
        )

    parser.add_argument(
        '-m', '--ncols',
        type=int, 
        default=25, 
        help='number of columns'
        )

    parser.add_argument(
        '-p', '--density',
        type=float, 
        default=0.2,
        help='initial density')
    
    parser.add_argument(
        '--nsteps', 
        type=int, 
        default=10, 
        help='steps to display'
        )

    args = parser.parse_args()
    
    main(n=args.n, m=args.m, p=args.p, nsteps=args.nsteps)
```
    

In [ ]:
# import importlib
# importlib.reload(conway)

In [ ]:
import conway

out1 = Output()
display(out1)

with out1:
    conway.main(clear_screen=lambda n: out1.clear_output(True))

Az `argparse` könyvtár nem a legkényelmesebb. Alternatívák:

- [Docopt](http://docopt.org/)
- [Click](https://pypi.org/project/click/)
- [clize](https://github.com/epsy/clize)

és még sok másik is!

## Itt is használhattunk volna dekorátort


A `ConwayGoL` példában utólag adtunk metódusokat az osztályunkhoz. Ezt is megtehettük volna dekorátorral.  

In [ ]:
def conway_method(f):
    setattr(ConwayGoL, f.__name__, f)
    return f


@conway_method
def dummy_method(self):
    print("this is a message from the new method!")

c = ConwayGoL([], 0)
c.dummy_method()

Azt is megtehettük volna, hogy a osztály nincs beleégetve a kódba.

In [ ]:
def new_method(cls):
    def decorator(f):
        setattr(cls, f.__name__, f)
        return f 
    return decorator

@new_method(ConwayGoL)
def dummy_method(self):
    print("Note that the old value of dummy_method is overwritten!")

In [ ]:
c.dummy_method()

# `with` statement


 
## Mi történik a `with` statement alkalmazásakor?

### Mit lehet a with mögé írni?

Hasonlóan a `for`-hoz szinte bármit, aminek van két metódusa:

- `__enter__`
- `__exit__`


    with obj as x:
       do_something with x

Itt az `obj.__enter__()` hívás eredménye lesz az x értéke és a block végén **GARANTÁLTAN** végrehajtódik az `obj.__exit__(...)` hívás.
Az `__enter__` metódus végezheti az előkészítést, az `__exit__` a takarítást!

#### Kell-e nekünk ezeket a metódusokat közvetlenül implementálni?

Valójában nem. Elegendő egy generátor függvényt megírni:

    import time
    
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
            

Így még csak egy generátort kapunk ami egyszer visszaad semmit (`None`) majd jelzi, hogy vége van a sorozatnak. 

Próbáljuk ki `for`-ral

In [ ]:
import time
    
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")
        
for x in timer():
    print(x)

`with`-del hibát kapunk

In [ ]:
with timer() as x:
    print(x)

A `contextlib` module `contextmanager` függvénye generátorból `contextmanager`-t készít.

In [ ]:
from contextlib import contextmanager
timer2 = contextmanager(timer)
with timer2() as x:
    print(x)

Ha egy függvény akarunk alkalmazni egy függvényre, hogy azt átalakítsuk, de ugyanaz maradjon a neve akkor a python `@` szintakszist használja. Ilyenkor ,,dekoráljuk'' a függvényt. A `contextmanager` egy példa **dekorátor**ra.

    @contextmanager
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
    
Példaként írjunk egy olyan contextmanager-t, ami jelzi nekünk, hogy mikor milyen hívás történik.

In [ ]:
@contextmanager
def print_whats_going_on(x):
    print("try blokk előtt")
    try:
        print("yield előtt")
        yield x
        print("yield után")
    except:
        print("except ág")
    finally:
        print("finally ág")
    print("try blokk után")

In [ ]:
with print_whats_going_on("hello") as x:
    print(x)
print("with után")

print("-"*50)

with print_whats_going_on("hello") as x:
    print(x)
    raise ValueError
print("with után")


In [ ]:
@contextmanager
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")

In [ ]:
t = timer()
with t:
    time.sleep(0.5)

Használhatjuk-e többször a `timer`-ünket?

In [ ]:
t = timer()
with t:
    time.sleep(0.5)
with t:
    time.sleep(0.5)


## Tudunk-e olyan contextmanagert készíteni, ami többször felhasználható?

pl.

```
with indented:
    indented.print("első")
    with indented:
        indented.print("második")
    indented.print("harmadik")
```

ahol kimenetként azt várnánk, hogy
```
  első
    második
  harmadik
```

In [ ]:
class Indent:
    def __init__(self, indent='  '):
        self.level = 0
        self.indent = indent
    
    def __enter__(self):
        self.level += 1
        return None
    
    def __exit__(self, *args):
        self.level -= 1
    
    def print(self, x):
        indent = self.indent*self.level 
        x = str(x)
        print('\n'.join(indent+line for line in x.split('\n')))


In [ ]:
indented = Indent('...')
text = """Többsoros szöveg:
Első
Második
"""
with indented:
    indented.print("első")
    with indented:
        indented.print("második")
        with indented:
            indented.print(text)
    indented.print("harmadik")